In [1]:
import os
import matplotlib.pyplot as plt 

print(os.getcwd())#显示当前路径
os.chdir('D:/bio/SPACEL/SPACEL-main')#更改路径，''里面为更改的路径
print(os.getcwd())#显示当前路径

D:\bio\jupyter_code\Saggital
D:\bio\SPACEL\SPACEL-main


In [ ]:
# datasets = ['Anterior1', 'Anterior2']
# datasets = ['Posterior1', 'Posterior2']
# datasets = ['Anterior1', 'Posterior1']
datasets = ['Anterior2', 'Posterior2']
sample_name = 'sample2'
input_dir = 'G:/dataset/2_saggital/input/'
output_dir = 'G:/dataset/2_saggital/output/SPACEL/'

In [ ]:
import SPACEL.Splane.utils
import pandas as pd
import scanpy as sc

for dataset in datasets:
    adata = sc.read_visium(input_dir + dataset)
    adata.var_names_make_unique()
    adata.obs_names_make_unique()
    adata_label = pd.read_csv(input_dir + dataset + '/truth.csv', index_col=0)
    adata_label.index = [str.replace(str.replace(x, '.', '-'), adata_label['batch'][0] + '-', '') for x in adata_label.index]
    adata_label = adata_label[['batch', 'celltype']]
    adata_label.rename(columns={'celltype': 'spatial_domain'}, inplace=True)
    SPACEL.Splane.utils.add_cell_type_composition(adata, adata_label)
    adata.uns['celltypes'] = adata.uns['celltypes'].tolist()
    print(adata.isbacked)
    adata.filename = output_dir + 'SPACEL_' + sample_name + '_' + dataset + '.h5ad'
    print(adata.isbacked)


# adata = sc.read_h5ad('D:/dataset/151508/SPACEL_151508.h5ad')
# print(adata)
# print(adata.obs)


In [ ]:
import scanpy as sc
import pandas as pd
adata_list = []
for dataset in datasets:
    path = output_dir + 'SPACEL_' + sample_name + '_' + dataset + '.h5ad'
    print(path)
    adata_list.append(sc.read_h5ad(path))


## Pairwise alignment
Here, we perform pairwise alignment of slices. The cluster_key is a column of .obs which contain spatial domain information used for alignment. The n_neighbors affect how much neighbors are considered when calculate similarity of two slices. The knn_exclude_cutoff affect max distance of neighbors. By default, knn_exclude_cutoff = n_neighbors, which means a spot/cell in source slices consider knn_exclude_cutoff neighbors in target slices, and neighbor will be exclude when the distance of neighbors larger than the median of neareast knn_exclude_cutoff + n_neighbors neighbors distance in all spot/cell in target slice. p affect the exponent of overlap penalty, and a larger p means stronger partial alignment capability.

In [ ]:
from SPACEL import Scube
Scube.align(adata_list,
  cluster_key='spatial_domain', 
  n_neighbors = 4, 
  p=1,
  write_loc_path = output_dir + sample_name + '_aligned_coordinates.csv'
 )

# sc.concat(adata_list).write('../data/ST_mouse_brain/mouse_brain_st.h5ad')

In [ ]:
print(adata_list[0])
print(adata_list[1])

In [ ]:
Scube.plot_stacked_slices(adata_list,spatial_key='spatial',cluster_key='spatial_domain')
plt.savefig(output_dir + sample_name + "_spatial.png", dpi=300)

In [ ]:
Scube.plot_stacked_slices(adata_list,spatial_key='spatial_pair',cluster_key='spatial_domain')
plt.savefig(output_dir + sample_name + "_spatial_pair.png", dpi=300)

In [ ]:
Scube.plot_stacked_slices(adata_list,spatial_key='spatial_aligned',cluster_key='spatial_domain')
plt.savefig(output_dir + sample_name + "_spatial_aligned.png", dpi=300)

In [ ]:
import seaborn as sns
import matplotlib
types = adata_list[0].obs['spatial_domain'].unique().tolist()
print(types)
print(len(types))
if len(types) > 10:
    colors = [matplotlib.colors.to_hex(c) for c in sns.color_palette('tab20',n_colors=len(types))]
else:
    colors = [matplotlib.colors.to_hex(c) for c in sns.color_palette('tab10',n_colors=len(types))]
print(types)
print(colors)

In [ ]:
coo = pd.DataFrame()
for i in range(len(adata_list)):
    loc = adata_list[i].obsm['spatial_aligned'].copy()
    loc['Z'] = i
    loc['celltype_colors'] = adata_list[i].obs['spatial_domain'].replace(dict(zip(types,colors)))
    coo = pd.concat([coo,loc],axis=0)
path = output_dir + sample_name + "_spatial_aligned_3d.png"
Scube.plot_3d(coo.values[:,:3],val=None,color=coo['celltype_colors'],show=True,elev=20,azim=180,s=1,save_path=path, save_dpi=300)

In [ ]:
for i in range(len(adata_list)):
    adata = adata_list[i]
    dataset = datasets[i]
    print(adata.isbacked)
    adata.filename = output_dir + 'SPACEL_aligned_' + sample_name + '_' + dataset + '.h5ad'
    print(adata.isbacked)